In [1]:
import torch
import torch.nn as nn

In [27]:
voxel = torch.randn(1,2,3,3,3) # (batch_size, channel, H, W, depth)
voxel

tensor([[[[[ 0.5707,  1.7499,  0.4102],
           [ 0.1635, -0.0988, -0.0264],
           [ 2.6499, -0.1418, -2.1186]],

          [[ 0.0049, -0.1285, -0.9183],
           [ 0.1833,  0.5092,  1.6888],
           [ 1.2434,  0.6154, -0.2424]],

          [[-0.8407, -0.1084, -1.2854],
           [-0.0700, -0.3714, -0.0925],
           [-1.6710,  0.3054, -0.7997]]],


         [[[-2.4083,  0.5959, -0.8859],
           [-0.5961,  2.0056, -2.4234],
           [ 0.5245, -0.7293,  0.3767]],

          [[-1.1699, -0.4419, -0.2192],
           [-1.2190, -2.0901, -0.0006],
           [ 0.5495,  1.6112, -1.7551]],

          [[-0.3411, -0.0840, -0.7943],
           [ 0.4908, -0.9687,  0.0815],
           [ 0.4169, -0.8095,  0.0221]]]]])

In [30]:
def soft_argmax(voxels):
    """
    Arguments: voxel patch in shape (batch_size, channel, H, W, depth)
    Return: 3D coordinates in shape (batch_size, channel, 3)
    """
    assert voxels.dim()==5
    # alpha is here to make the largest element really big, so it
    # would become very close to 1 after softmax
    alpha = 1000.0 
    N,C,H,W,D = voxels.shape
    soft_max = nn.functional.softmax(voxels.view(N,C,-1)*alpha,dim=2)
    soft_max = soft_max.view(voxels.shape)
    indices_kernel = torch.arange(start=0,end=H*W*D).unsqueeze(0)
    indices_kernel = indices_kernel.view((H,W,D))
    conv = soft_max*indices_kernel
    indices = conv.sum(2).sum(2).sum(2)
    z = indices%D
    y = (indices/D).floor()%W
    x = (((indices/D).floor())/W).floor()%H
    coords = torch.stack([x,y,z],dim=2)
    return coords

In [31]:
coords = soft_argmax(voxel)
coords

tensor([[[ 0.,  2.,  0.],
         [ 0.,  1.,  1.]]])